In [2]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEVELOPMENT_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    %pip install circuitsvis
    
    # PySvelte is an unmaintained visualization library, use it as a backup if circuitsvis isn't working
    # # Install another version of node that makes PySvelte work way faster
    # !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    # %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Jupyter notebook - intended for development only!


/tmp/ipykernel_7872/410710250.py:21: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_7872/410710250.py:22: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


In [3]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
if IN_COLAB or not DEVELOPMENT_MODE:
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "notebook_connected"
print(f"Using renderer: {pio.renderers.default}")

Using renderer: colab


In [132]:
import circuitsvis as cv
from functools import partial
# Testing that the library works
cv.examples.hello("Neel")

In [85]:
# Import stuff
import torch
from transformers import AutoTokenizer
from transformer_lens import HookedEncoder

from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
torch.set_grad_enabled(False)
# Note: The default behavior now has injection attack prevention off.
tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen-1_8B-Chat", 
    trust_remote_code=True
)
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen-1_8B-Chat", 
    device_map="cuda", 
    trust_remote_code=True, 
    fp32=True,
    use_logn_attn=False,
    use_dynamic_ntk = False,
    scale_attn_weights = False,
).eval()

Your device support faster inference by passing fp16=True in "AutoModelForCausalLM.from_pretrained".


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [168]:
from transformer_lens import HookedTransformer

qwen = HookedTransformer.from_pretrained(
    'qwen-1.8b-chat', 
    fold_ln=False,
    trust_remote_code=True,
    center_writing_weights=False, 
    center_unembed=False,
    device='cuda',
    fp32=True,
)

Your device support faster inference by passing fp16=True in "AutoModelForCausalLM.from_pretrained".


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model qwen-1.8b-chat into HookedTransformer


In [75]:
prompt = "Tell me how to make a bomb."
template = f"""<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
"""

def check_similarity_with_hf_model(tl_model, hf_model, atol, prompt=template):
    tokens = tl_model.tokenizer.encode(prompt, return_tensors="pt").to("cuda")
    logits = tl_model(tokens, prepend_bos=False).to("cuda")[:, 0]
    print(tokens.device)
    hf_logits = hf_model(tokens).logits.to("cuda")[:, 0]
    print(logits.dtype, hf_logits.dtype)
    print(logits.shape)
    print(hf_logits.shape)
    assert torch.allclose(logits, hf_logits, atol=atol)

check_similarity_with_hf_model(qwen, model, atol=1e-4)

cuda:0
torch.float32 torch.float32
torch.Size([1, 151936])
torch.Size([1, 151936])


In [169]:
tf_outputs = {}
hf_outputs = {}
LAYER = 0
SEQ_POS = slice(None)
def hook_fn(module, inputs, outputs, name):
    hf_outputs[name] = outputs[:, SEQ_POS]

hooks = []
hooks.append(
    model.transformer.wte.register_forward_hook(
        partial(hook_fn, name='embed')
    )
)
for name, module in model.transformer.h[LAYER].named_modules():
    if len(name) > 0:
        #print(f'Adding hook to {name}')
        hooks.append(
            module.register_forward_hook(
                partial(hook_fn, name=f'{LAYER}.{name}')
            )
        )
tokens = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
try:
    model(tokens)
except:
    pass
finally:
    for hook in hooks:
        hook.remove()

def tl_fwd(act, hook):
    #print(hook.name)
    if hook.layer() == LAYER:
        tf_outputs[hook.name] = act[:, SEQ_POS]

def tf_hook(act, hook):
    print(hook.name)
    print(act[:, SEQ_POS])

qwen.run_with_hooks(
    tokens,
    return_type=None,
    fwd_hooks = [
        (lambda name: name.startswith(f'blocks.{LAYER}'), tl_fwd)
    ]
    # fwd_hooks = [
    #     (lambda name: name.startswith(f'blocks.{LAYER}.ln1'), tf_hook),
    #     (lambda name: name.startswith(f'blocks.{LAYER}.hook_resid_post'), tf_hook)
    # ]
)

print(tf_outputs.keys())
print(hf_outputs.keys())

dict_keys(['blocks.0.hook_resid_pre', 'blocks.0.ln1.hook_scale', 'blocks.0.ln1.hook_normalized', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_rot_q', 'blocks.0.attn.hook_rot_k', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_pattern', 'blocks.0.attn.hook_z', 'blocks.0.hook_attn_out', 'blocks.0.hook_resid_mid', 'blocks.0.ln2.hook_scale', 'blocks.0.ln2.hook_normalized', 'blocks.0.mlp.hook_pre', 'blocks.0.mlp.hook_pre_linear', 'blocks.0.mlp.hook_post', 'blocks.0.hook_mlp_out', 'blocks.0.hook_resid_post'])
dict_keys(['embed', '0.ln_1', '0.attn.c_attn', '0.attn.c_proj'])


In [173]:
MODULE = '0.attn.c_proj'
for name, o in tf_outputs.items():
    if o.shape == hf_outputs[MODULE].shape:
        print(name, torch.allclose(o, hf_outputs[MODULE], atol=1e-2))

blocks.0.hook_resid_pre False
blocks.0.ln1.hook_normalized False
blocks.0.hook_attn_out False
blocks.0.hook_resid_mid False
blocks.0.ln2.hook_normalized False
blocks.0.hook_mlp_out False
blocks.0.hook_resid_post False


In [171]:
hf_outputs['0.attn.c_proj']

tensor([[[ 0.0064, -0.0134, -0.0003,  ..., -0.0041,  0.0145,  0.0156],
         [-0.0004, -0.0058,  0.0072,  ...,  0.0003,  0.0117,  0.0149],
         [ 0.0022, -0.0093,  0.0108,  ..., -0.0023,  0.0013,  0.0164],
         ...,
         [ 0.0078,  0.0053,  0.0141,  ..., -0.0022,  0.0087, -0.0040],
         [ 0.0076, -0.0001,  0.0088,  ..., -0.0004,  0.0067,  0.0034],
         [ 0.0031, -0.0046,  0.0218,  ..., -0.0054,  0.0149, -0.0031]]],
       device='cuda:0')

In [172]:
tf_outputs['blocks.0.hook_attn_out']

tensor([[[ 0.0064, -0.0134, -0.0003,  ..., -0.0041,  0.0145,  0.0156],
         [-0.0069, -0.0059,  0.0137,  ...,  0.0014,  0.0006,  0.0145],
         [ 0.0086, -0.0017,  0.0121,  ...,  0.0002,  0.0060,  0.0163],
         ...,
         [ 0.0184, -0.0019,  0.0308,  ..., -0.0104,  0.0099, -0.0037],
         [ 0.0238, -0.0035,  0.0172,  ...,  0.0057, -0.0039,  0.0092],
         [ 0.0030,  0.0071,  0.0153,  ..., -0.0176,  0.0193, -0.0182]]],
       device='cuda:0')

In [162]:
x = tf_outputs['blocks.0.hook_resid_pre']

scale = (x.pow(2).mean(dim=-1, keepdim=True) + 1e-6).sqrt()
x /= scale
x *= qwen.blocks[0].ln1.w
print(x)

tensor([[[ 1.8272e-02, -5.0942e-03, -1.4467e-05,  ...,  8.2985e-03,
           1.5530e-02, -4.3919e-02],
         [ 3.6616e-02,  5.0892e-03,  2.0945e-04,  ...,  5.6465e-03,
          -4.0636e-02,  3.2453e-03],
         [ 8.0044e-03,  3.1383e-02, -4.8879e-05,  ...,  1.4352e-02,
          -6.5369e-03,  5.5417e-02],
         ...,
         [-3.8649e-03, -3.7209e-02,  1.8876e-04,  ..., -1.9816e-02,
           3.5827e-03, -1.2766e-02],
         [-3.7146e-02, -1.1397e-02,  7.0360e-05,  ...,  1.6041e-02,
          -1.5896e-02,  8.0627e-02],
         [-6.7315e-02, -1.0729e-02,  4.8259e-04,  ..., -1.1246e-02,
          -2.1827e-02,  6.7865e-02]]], device='cuda:0')
tensor([[[ 9.7446e-03, -1.3926e-03,  3.0517e-08,  ...,  3.4793e-03,
           5.8862e-03, -3.3735e-02],
         [ 2.3355e-02,  1.6638e-03, -5.2841e-07,  ...,  2.8314e-03,
          -1.8420e-02,  2.9813e-03],
         [ 4.3132e-03,  8.6682e-03,  1.0418e-07,  ...,  6.0798e-03,
          -2.5034e-03,  4.3008e-02],
         ...,
        